In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP']
plt.rcParams['font.sans-serif'] = ['Noto Serif CJK JP']

对每个BMI分组，最小化平均达标时间，最大化达标率

1. ~~通过聚类确定分组个数~~
2. 优化分组最小化平均达标时间

In [30]:
data_m = pd.read_excel('附件.xlsx', index_col=0, sheet_name='男胎检测数据')

In [31]:
week_day = data_m['检测孕周'].str.lower().str.split('w')
# data_m.drop('检测孕周', axis=1, inplace=True)
data_m['检测孕周-周'] = week_day.apply(lambda x: int(x[0]))
data_m['检测孕周-天'] = week_day.apply(lambda x: int(0 if x[1] == '' else x[1]))
data_m['检测孕周'] = data_m['检测孕周-周'] + data_m['检测孕周-天'] / 7
data_m['孕妇BMI-分组'] = pd.cut(data_m['孕妇BMI'], bins=[20, 28, 32, 36, 40, np.inf], right=False).values.astype(str)
data_m['胎儿是否健康'] = data_m['胎儿是否健康'].map({'是': 1, '否': 0})
data_m['染色体的非整倍体-无异常'] = data_m['染色体的非整倍体'].isna().astype(int)
data_m['染色体的非整倍体-T13异常'] = data_m['染色体的非整倍体'].apply(lambda x: pd.notna(x) and 'T13' in x).astype(int)
data_m['染色体的非整倍体-T18异常'] = data_m['染色体的非整倍体'].apply(lambda x: pd.notna(x) and 'T18' in x).astype(int)
data_m['染色体的非整倍体-T21异常'] = data_m['染色体的非整倍体'].apply(lambda x: pd.notna(x) and 'T21' in x).astype(int)
data_m.drop(columns=['染色体的非整倍体'], inplace=True)
data_m = data_m.rename(columns={'唯一比对的读段数  ': '唯一比对的读段数'})
data_m['达标'] = (data_m['Y染色体浓度'] > .04).astype(int)
# data_m = data_m[data_m['检测孕周-周'] >= 10]
data_m

,孕妇代码,年龄,身高,体重,末次月经,IVF妊娠,检测日期,检测抽血次数,检测孕周,孕妇BMI,...,生产次数,胎儿是否健康,检测孕周-周,检测孕周-天,孕妇BMI-分组,染色体的非整倍体-无异常,染色体的非整倍体-T13异常,染色体的非整倍体-T18异常,染色体的非整倍体-T21异常,达标
序号,,,,,,,,,,,,,,,,,,,,,
1,A001,31,160.0,72.00,2023-02-01 00:00:00,自然受孕,20230429,1,11.857143,28.125000,...,0,1,11,6,"[28.0, 32.0)",1,0,0,0,0
2,A001,31,160.0,73.00,2023-02-01 00:00:00,自然受孕,20230531,2,15.857143,28.515625,...,0,1,15,6,"[28.0, 32.0)",1,0,0,0,0
3,A001,31,160.0,73.00,2023-02-01 00:00:00,自然受孕,20230625,3,20.142857,28.515625,...,0,1,20,1,"[28.0, 32.0)",1,0,0,0,1
4,A001,31,160.0,74.00,2023-02-01 00:00:00,自然受孕,20230716,4,22.857143,28.906250,...,0,1,22,6,"[28.0, 32.0)",1,0,0,0,1
5,A002,32,149.0,74.00,2023-11-09 00:00:00,自然受孕,20240219,1,13.857143,33.331832,...,1,0,13,6,"[32.0, 36.0)",1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,A266,30,159.0,83.35,2022-12-29,自然受孕,2023-05-02 00:00:00,4,17.714286,32.969881,...,0,1,17,5,"[32.0, 36.0)",0,0,1,0,1
1079,A267,28,155.0,73.76,2023-02-25,自然受孕,2023-05-17 00:00:00,1,11.571429,30.703133,...,0,1,11,4,"[28.0, 32.0)",0,0,0,1,1
1080,A267,28,155.0,74.06,2023-02-25,自然受孕,2023-05-24 00:00:00,2,12.571429,30.825814,...,0,1,12,4,"[28.0, 32.0)",1,0,0,0,1


In [32]:
mean_avg_first_achieve_time = 0
mean_achievement_rate = 0
for group, df in data_m.groupby(['孕妇BMI-分组']):
    # 平均达标时间
    first_achieve_time = df[df['达标'] == 1].groupby('孕妇代码')['检测孕周-周'].first()
    avg_first_achieve_time = first_achieve_time.mean()
    # 达标率
    all_codes = df['孕妇代码'].unique()
    total_codes = len(all_codes)
    achieved_codes = len(first_achieve_time)
    achievement_rate = achieved_codes / total_codes if total_codes > 0 else 0
    print('平均达标时间:', avg_first_achieve_time, '达标率:', achievement_rate)
    mean_avg_first_achieve_time += avg_first_achieve_time
    mean_achievement_rate += achievement_rate
mean_avg_first_achieve_time /= len(data_m['孕妇BMI-分组'].unique())
mean_achievement_rate /= len(data_m['孕妇BMI-分组'].unique())
print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)

平均达标时间: 15.142857142857142 达标率: 0.875
平均达标时间: 13.236486486486486 达标率: 0.9801324503311258
平均达标时间: 14.296610169491526 达标率: 0.9365079365079365
平均达标时间: 17.9 达标率: 0.967741935483871
平均达标时间: 19.285714285714285 达标率: 0.875
平均达标时间均值: 15.972333616909888 平均达标率: 0.9268764644645866


In [99]:
def calc_mean_metric(data_m, show_print=False):
    mean_avg_first_achieve_time = 0
    mean_med_first_achieve_time = 0
    mean_achievement_rate = 0
    cnt = 0
    for _, df in data_m.groupby(['孕妇BMI-分组']):
        # print(len(df))
        # 平均达标时间
        first_achieve_time = df[df['达标'] == 1].groupby('孕妇代码')['检测孕周-周'].first()
        avg_first_achieve_time = first_achieve_time.mean()
        med_first_achieve_time = first_achieve_time.median()
        # 达标率
        all_codes = df['孕妇代码'].unique()
        total_codes = len(all_codes)
        achieved_codes = len(first_achieve_time)
        achievement_rate = achieved_codes / total_codes if total_codes > 0 else 0
        if achievement_rate == 0:
            continue
        if show_print:
            print(
                '分组:', _,
                '平均达标时间:', avg_first_achieve_time,
                '中位达标时间:', med_first_achieve_time,
                '达标率:', achievement_rate,
            )
        cnt += 1
        mean_avg_first_achieve_time += avg_first_achieve_time
        mean_med_first_achieve_time += med_first_achieve_time
        mean_achievement_rate += achievement_rate
    mean_avg_first_achieve_time /= cnt
    mean_achievement_rate /= cnt
    # print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)
    return mean_avg_first_achieve_time, mean_achievement_rate

In [100]:
n_dim = 4
temp = np.linspace(data_m['孕妇BMI'].min(), data_m['孕妇BMI'].max(), n_dim)
temp = np.append(np.append(temp, -np.inf), np.inf)
temp = np.unique(temp)
temp = np.sort(temp)
data_m['孕妇BMI-分组'] = pd.cut(data_m['孕妇BMI'], temp, right=False).values.astype(str)
display(data_m[['孕妇代码', '孕妇BMI', '孕妇BMI-分组']])
print(data_m['孕妇BMI-分组'].unique())
mean_avg_first_achieve_time, mean_achievement_rate = calc_mean_metric(data_m, True)
print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)

,孕妇代码,孕妇BMI,孕妇BMI-分组
序号,,,
1,A001,28.125000,"[20.703, 29.427)"
2,A001,28.515625,"[20.703, 29.427)"
3,A001,28.515625,"[20.703, 29.427)"
4,A001,28.906250,"[20.703, 29.427)"
5,A002,33.331832,"[29.427, 38.151)"
...,...,...,...
1078,A266,32.969881,"[29.427, 38.151)"
1079,A267,30.703133,"[29.427, 38.151)"
1080,A267,30.825814,"[29.427, 38.151)"


['[20.703, 29.427)' '[29.427, 38.151)' '[38.151, 46.875)' '[46.875, inf)']
分组: ('[20.703, 29.427)',) 平均达标时间: 13.228070175438596 中位达标时间: 13.0 达标率: 0.95
分组: ('[29.427, 38.151)',) 平均达标时间: 13.79646017699115 中位达标时间: 13.0 达标率: 0.9699570815450643
分组: ('[38.151, 46.875)',) 平均达标时间: 17.0 中位达标时间: 16.0 达标率: 0.9285714285714286
平均达标时间均值: 14.674843450809917 平均达标率: 0.9495095033721643


In [101]:
data_m['孕妇BMI-分组'] = pd.cut(data_m['孕妇BMI'], bins=[20, 28, 32, 36, 40, np.inf], right=False).values.astype(str)
display(data_m[['孕妇代码', '孕妇BMI', '孕妇BMI-分组']])
print(data_m['孕妇BMI-分组'].unique())
mean_avg_first_achieve_time, mean_achievement_rate = calc_mean_metric(data_m, True)
print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)

,孕妇代码,孕妇BMI,孕妇BMI-分组
序号,,,
1,A001,28.125000,"[28.0, 32.0)"
2,A001,28.515625,"[28.0, 32.0)"
3,A001,28.515625,"[28.0, 32.0)"
4,A001,28.906250,"[28.0, 32.0)"
5,A002,33.331832,"[32.0, 36.0)"
...,...,...,...
1078,A266,32.969881,"[32.0, 36.0)"
1079,A267,30.703133,"[28.0, 32.0)"
1080,A267,30.825814,"[28.0, 32.0)"


['[28.0, 32.0)' '[32.0, 36.0)' '[36.0, 40.0)' '[20.0, 28.0)' '[40.0, inf)']
分组: ('[20.0, 28.0)',) 平均达标时间: 15.142857142857142 中位达标时间: 13.0 达标率: 0.875
分组: ('[28.0, 32.0)',) 平均达标时间: 13.236486486486486 中位达标时间: 12.0 达标率: 0.9801324503311258
分组: ('[32.0, 36.0)',) 平均达标时间: 14.296610169491526 中位达标时间: 13.0 达标率: 0.9365079365079365
分组: ('[36.0, 40.0)',) 平均达标时间: 17.9 中位达标时间: 16.5 达标率: 0.967741935483871
分组: ('[40.0, inf)',) 平均达标时间: 19.285714285714285 中位达标时间: 20.0 达标率: 0.875
平均达标时间均值: 15.972333616909888 平均达标率: 0.9268764644645866


In [43]:
def obj(x):
    x = np.append(np.append(x, -np.inf), np.inf)
    x = np.sort(np.unique(x))
    data_m['孕妇BMI-分组'] = pd.cut(data_m['孕妇BMI'], x, right=False).values.astype(str)
    mean_avg_first_achieve_time, mean_achievement_rate = calc_mean_metric(data_m)
    return mean_avg_first_achieve_time

In [69]:
data_m['孕妇BMI'].min(), data_m['孕妇BMI'].max()

(20.703125, 46.875)

In [74]:
obj(np.array([[11]])), obj(np.array([[21]])), obj(np.array([[31]])), obj(np.array([[41]])), obj(np.array([[51]]))

(13.203846153846154,
 14.596525096525095,
 14.06236559139785,
 16.554474708171206,
 13.203846153846154)

In [91]:
from sko.GA import GA
from sko.tool_kit import x2gray

n_dim = 7  # 中国BMI分类参考标准
size_pop = 200
x0 = np.linspace(data_m['孕妇BMI'].min()+0.1, data_m['孕妇BMI'].max()-0.1, n_dim).reshape(-1, n_dim)
x0 = np.array([[21]])
ga = GA(
    func=lambda *x: obj(x),
    n_dim=n_dim,
    size_pop=size_pop,
    max_iter=400,
    prob_mut=0.001,
    lb=[data_m['孕妇BMI'].min()] * n_dim,
    ub=[data_m['孕妇BMI'].max()] * n_dim,
    constraint_eq=tuple(),
    constraint_ueq=tuple(),
    precision=1e-07,
    early_stop=True,
)
ga.Chrom[0] = x2gray(x0, n_dim, ga.lb, ga.ub, ga.precision)
xbest, ybest = ga.run()
print(xbest, ybest)

[31.01035931 22.31409758 22.53315356 26.68613145 45.8813175  26.43571948
 33.16273491] [14.34240582]


In [102]:
x = np.array([-np.inf, 18.5, 24, 28, 32, 36, 40, np.inf])
data_m['孕妇BMI-分组'] = pd.cut(data_m['孕妇BMI'], x, right=False).values.astype(str)
display(data_m[['孕妇代码', '孕妇BMI', '孕妇BMI-分组']])
print(data_m['孕妇BMI-分组'].unique())
mean_avg_first_achieve_time, mean_achievement_rate = calc_mean_metric(data_m, True)
print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)

,孕妇代码,孕妇BMI,孕妇BMI-分组
序号,,,
1,A001,28.125000,"[28.0, 32.0)"
2,A001,28.515625,"[28.0, 32.0)"
3,A001,28.515625,"[28.0, 32.0)"
4,A001,28.906250,"[28.0, 32.0)"
5,A002,33.331832,"[32.0, 36.0)"
...,...,...,...
1078,A266,32.969881,"[32.0, 36.0)"
1079,A267,30.703133,"[28.0, 32.0)"
1080,A267,30.825814,"[28.0, 32.0)"


['[28.0, 32.0)' '[32.0, 36.0)' '[36.0, 40.0)' '[24.0, 28.0)' '[40.0, inf)'
 '[18.5, 24.0)']
分组: ('[18.5, 24.0)',) 平均达标时间: 16.0 中位达标时间: 16.0 达标率: 1.0
分组: ('[24.0, 28.0)',) 平均达标时间: 15.0 中位达标时间: 12.5 达标率: 0.8571428571428571
分组: ('[28.0, 32.0)',) 平均达标时间: 13.236486486486486 中位达标时间: 12.0 达标率: 0.9801324503311258
分组: ('[32.0, 36.0)',) 平均达标时间: 14.296610169491526 中位达标时间: 13.0 达标率: 0.9365079365079365
分组: ('[36.0, 40.0)',) 平均达标时间: 17.9 中位达标时间: 16.5 达标率: 0.967741935483871
分组: ('[40.0, inf)',) 平均达标时间: 19.285714285714285 中位达标时间: 20.0 达标率: 0.875
平均达标时间均值: 15.953135156948713 平均达标率: 0.9360875299109651


In [104]:
# x = np.array([-np.inf, 18.5, 24, 28, 32, 36, 40, np.inf])
data_m['孕妇BMI-分组'] = pd.qcut(data_m['孕妇BMI'], q=6).values.astype(str)
display(data_m[['孕妇代码', '孕妇BMI', '孕妇BMI-分组']])
print(data_m['孕妇BMI-分组'].unique())
mean_avg_first_achieve_time, mean_achievement_rate = calc_mean_metric(data_m, True)
print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)

,孕妇代码,孕妇BMI,孕妇BMI-分组
序号,,,
1,A001,28.125000,"(20.701999999999998, 29.688]"
2,A001,28.515625,"(20.701999999999998, 29.688]"
3,A001,28.515625,"(20.701999999999998, 29.688]"
4,A001,28.906250,"(20.701999999999998, 29.688]"
5,A002,33.331832,"(33.216, 34.781]"
...,...,...,...
1078,A266,32.969881,"(31.812, 33.216]"
1079,A267,30.703133,"(29.688, 30.71]"
1080,A267,30.825814,"(30.71, 31.812]"


['(20.701999999999998, 29.688]' '(33.216, 34.781]' '(30.71, 31.812]'
 '(31.812, 33.216]' '(29.688, 30.71]' '(34.781, 46.875]']
分组: ('(20.701999999999998, 29.688]',) 平均达标时间: 13.246153846153845 中位达标时间: 12.0 达标率: 0.9420289855072463
分组: ('(29.688, 30.71]',) 平均达标时间: 14.987951807228916 中位达标时间: 13.0 达标率: 0.9325842696629213
分组: ('(30.71, 31.812]',) 平均达标时间: 16.346153846153847 中位达标时间: 15.0 达标率: 0.9629629629629629
分组: ('(31.812, 33.216]',) 平均达标时间: 15.222222222222221 中位达标时间: 13.0 达标率: 0.9230769230769231
分组: ('(33.216, 34.781]',) 平均达标时间: 15.090909090909092 中位达标时间: 14.0 达标率: 0.8918918918918919
分组: ('(34.781, 46.875]',) 平均达标时间: 16.22641509433962 中位达标时间: 15.0 达标率: 0.9464285714285714
平均达标时间均值: 15.186634317834589 平均达标率: 0.9331622674217529


In [103]:
# x = np.array([-np.inf, 18.5, 24, 28, 32, 36, 40, np.inf])
data_m['孕妇BMI-分组'] = pd.qcut(data_m['孕妇BMI'], q=7).values.astype(str)
display(data_m[['孕妇代码', '孕妇BMI', '孕妇BMI-分组']])
print(data_m['孕妇BMI-分组'].unique())
mean_avg_first_achieve_time, mean_achievement_rate = calc_mean_metric(data_m, True)
print('平均达标时间均值:', mean_avg_first_achieve_time, '平均达标率:', mean_achievement_rate)

,孕妇代码,孕妇BMI,孕妇BMI-分组
序号,,,
1,A001,28.125000,"(20.701999999999998, 29.444]"
2,A001,28.515625,"(20.701999999999998, 29.444]"
3,A001,28.515625,"(20.701999999999998, 29.444]"
4,A001,28.906250,"(20.701999999999998, 29.444]"
5,A002,33.331832,"(32.417, 33.557]"
...,...,...,...
1078,A266,32.969881,"(32.417, 33.557]"
1079,A267,30.703133,"(30.469, 31.25]"
1080,A267,30.825814,"(30.469, 31.25]"


['(20.701999999999998, 29.444]' '(32.417, 33.557]' '(33.557, 35.19]'
 '(30.469, 31.25]' '(31.25, 32.417]' '(29.444, 30.469]' '(35.19, 46.875]']
分组: ('(20.701999999999998, 29.444]',) 平均达标时间: 13.189655172413794 中位达标时间: 12.5 达标率: 0.9508196721311475
分组: ('(29.444, 30.469]',) 平均达标时间: 14.933333333333334 中位达标时间: 13.0 达标率: 0.9259259259259259
分组: ('(30.469, 31.25]',) 平均达标时间: 16.357142857142858 中位达标时间: 15.0 达标率: 0.9459459459459459
分组: ('(31.25, 32.417]',) 平均达标时间: 16.246376811594203 中位达标时间: 16.0 达标率: 0.9452054794520548
分组: ('(32.417, 33.557]',) 平均达标时间: 15.0625 中位达标时间: 13.0 达标率: 0.9014084507042254
分组: ('(33.557, 35.19]',) 平均达标时间: 15.0 中位达标时间: 13.0 达标率: 0.9166666666666666
分组: ('(35.19, 46.875]',) 平均达标时间: 16.47826086956522 中位达标时间: 16.0 达标率: 0.9583333333333334
平均达标时间均值: 15.323895577721345 平均达标率: 0.934900782022757
